In [1]:
from sklearn.decomposition import IncrementalPCA
import os
import sklearn
from skimage import io
import numpy as np 
import matplotlib.pyplot as plt
import cv2
import sys
sys.path.append('../')
import torch as tc
from src.dataset.dataloader import *


In [2]:
import importlib
importlib.reload(sys.modules['src.dataset.dataloader'])
from src.dataset.dataloader import *

In [3]:
img_dir = "../data/raw"
batch_size = 10000
size = 425000
dataset = CREDO_Dataset(img_dir, True)
dataloader = tc.utils.data.DataLoader(dataset, batch_size = batch_size)

In [4]:
import time
import pickle as pkl
dimensions = 30
#sklearn_pca = IncrementalPCA(n_components=dimensions)
sklearn_pca = pkl.load(open("../models/model_inc_pca.pkl",'rb'))
for index, data in enumerate(dataloader):
    if index > 12:
        time1 = time.time()

        x, y, z = data[0].shape
        data_chunk = data[0].reshape(x, y*z)

        time2 = time.time()

        print(f"Czas reshape: {time2-time1}")
        sklearn_pca.partial_fit(data_chunk)

        time3 = time.time()
        print(f"Czas PCA: {time3-time2}")

        pkl.dump(sklearn_pca, open("../models/model_inc_pca.pkl","wb"))

        print(f"DONE: {(index+1)*x}/{size}")

        #Estimated waiting time: ~40 minutes


Czas reshape: 0.012493133544921875
Czas PCA: 26.300886154174805
DONE: 140000/425000
Czas reshape: 0.018510818481445312
Czas PCA: 26.82866334915161
DONE: 150000/425000
Czas reshape: 0.009003639221191406
Czas PCA: 27.42433786392212
DONE: 160000/425000
Czas reshape: 0.016002655029296875
Czas PCA: 27.464696645736694
DONE: 170000/425000
Czas reshape: 0.011997699737548828
Czas PCA: 28.820557594299316
DONE: 180000/425000
Czas reshape: 0.009000301361083984
Czas PCA: 26.760088682174683
DONE: 190000/425000
Czas reshape: 0.008994102478027344
Czas PCA: 27.76287579536438
DONE: 200000/425000
Czas reshape: 0.008999824523925781
Czas PCA: 28.0314884185791
DONE: 210000/425000
Czas reshape: 0.013997554779052734
Czas PCA: 27.993238925933838
DONE: 220000/425000
Czas reshape: 0.010004281997680664
Czas PCA: 28.37176513671875
DONE: 230000/425000
Czas reshape: 0.009011983871459961
Czas PCA: 27.37360668182373
DONE: 240000/425000
Czas reshape: 0.009000301361083984
Czas PCA: 27.748070001602173
DONE: 250000/425000

In [6]:
print(sum(sklearn_pca.explained_variance_ratio_[:30]))

#8 cech wyjaśnia już aż 96,08% !

0.879322263561803
